# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import string as st
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import torch
# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Function

In [2]:
def preprocess_text(text):
    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
    # Remove stopwords, punctuation, and lemmatize
    tokens = [token.lemma_ for token in doc 
             if not token.is_stop and not token.is_punct and token.is_alpha]
    
    return ' '.join(tokens)

In [3]:
def extract_linguistic_features(text):
    doc = nlp(text)
    features = {
        'num_verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'num_nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'sentence_length': len(doc),
        'blooms_verb_present': any(token.text in {'analyze', 'evaluate', 'create'} for token in doc)
    }
    return features

# Pre-Processing

In [4]:
dataset_name = 'balanced_dataset'

# Load dataset
df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/' + dataset_name + '.csv')

# Apply preprocessing
df['processed_question'] = df['QUESTION'].apply(preprocess_text)

## Tokenize

BERT model is used to retain context

In [ ]:
# Load pre-trained BERT (TensorFlow version)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode
inputs = tokenizer(df['processed_question'].tolist(), return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
print("BERT Sentence Embedding Shape:", sentence_embedding.shape)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

BERT Sentence Embedding Shape: (600, 768)


In [5]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['processed_question'])

In [47]:
# Addeing linguistic features
ling_features = df['processed_question'].apply(extract_linguistic_features).apply(pd.Series)
token_df = pd.concat([ling_features, pd.DataFrame(sentence_embedding)], axis=1)
token_df = token_df.drop(columns= ['blooms_verb_present'] , axis=1) 
token_df.columns = token_df.columns.map(str)


In [6]:
y = df['BT LEVEL']

# Modelling

## Basic Machine Leanring

In [7]:
x_train , x_test , y_train, y_test = train_test_split(tfidf, y, test_size=0.2, random_state=34 , stratify= y)

### Random Forest

In [92]:
params = {
    'n_estimators' : [2 , 5 , 10 , 20 , 50 , 100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 20 , 50]
}
param_search = GridSearchCV(estimator= RandomForestClassifier(), param_grid= params, cv=5)

param_search.fit(tfidf , y)
print("Best Parameters: ", param_search.best_params_)

Best Parameters:  {'criterion': 'gini', 'max_depth': 50, 'n_estimators': 100}


In [8]:
classifier = RandomForestClassifier(n_estimators= 100 , max_depth= 50, criterion= 'gini')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     Analysis       0.90      0.95      0.93        20
  Application       0.74      0.70      0.72        20
Comprehension       1.00      0.70      0.82        20
   Evaluation       0.80      0.60      0.69        20
    Knowledge       0.55      0.85      0.67        20
    Synthesis       0.70      0.70      0.70        20

     accuracy                           0.75       120
    macro avg       0.78      0.75      0.75       120
 weighted avg       0.78      0.75      0.75       120



In [10]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
print(classifier.predict(tfidf_q))

['Evaluation']
